In [ ]:
!python --version
# %tensorflow_version 2.x

Python 3.10.12


In [ ]:
# !sudo apt-get install python3.7
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
# !sudo update-alternatives --config python3
# !sudo apt install python3-pip
# !python --version
# !sudo apt-get install python3.7-distutils

In [ ]:
# !pip install transformers==4.11.3
# !pip install transformers
# !pip install accelerate -U
!pip install transformers[torch] accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.2 MB/s eta 0:00:00


In [ ]:
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [ ]:
# !pip install transformers==4.11.3
!pip install torchaudio
!pip install librosa
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.3 MB/s eta 0:00:00


In [ ]:
!pip install ipywidgets==7.7.1
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.2 MB/s eta 0:00:00


In [ ]:
# login to huggingface using 'write' token type generated on your huggingface account

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# install huggingface datasets library for loading datasets

!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00


In [ ]:
# load common voice datasets for training and testing model

from datasets import load_dataset

training_data = load_dataset('mozilla-foundation/common_voice_13_0', 'sw', split='train+validation')
testing_data = load_dataset('mozilla-foundation/common_voice_13_0', 'sw', split='test')

# Load the 'mozilla-foundation/common_voice_13_0' dataset for Swahili ('sw')
# dataset = load_dataset('mozilla-foundation/common_voice_13_0', 'sw')

# Load a smaller portion (e.g., 10%) of the training and validation data
# training_data = dataset['train+validation'].shard(percentage=10)

# Load the entire test split
# testing_data = dataset['test'].shard(percentage=10)

In [ ]:
training_data

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 46254
})

In [ ]:
testing_data

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 11271
})

In [ ]:
# shard to reduce number of rows for training

# train_data = dataset['train'].shard(num_shards=5, index=0)
# validation_data = dataset['validation'].shard(num_shards=5, index=0)
# training_data = dataset['train+validation'].shard(num_shards=5, index=0)
# testing_data = dataset['test'].shard(num_shards=5, index=0)

training_data = training_data.shard(num_shards=5, index=0)
testing_data = testing_data.shard(num_shards=5, index=0)


In [ ]:
training_data

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 9251
})

In [ ]:
testing_data

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 2255
})

In [ ]:
# remove unnecessary columns as we don't need them.
# We only need 'path', 'audio' and 'sentence' columns for training

training_data = training_data.remove_columns(['accent', 'age', 'client_id', 'down_votes', 'up_votes', 'gender', 'locale', 'segment', 'variant'])
testing_data = testing_data.remove_columns(['accent', 'age', 'client_id', 'down_votes', 'up_votes', 'gender', 'locale', 'segment', 'variant'])

In [ ]:
training_data

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 9251
})

In [ ]:
testing_data

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 2255
})

In [ ]:
# Install Git-LFS to upload model's checkpoints:

!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [ ]:
# import some neccessary libraries

from datasets import load_metric, Audio, ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

# show_random_elements(training_data)

In [ ]:
# prepare function to remove special characters as we don't need them in our model

import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:
training_data = training_data.map(remove_special_characters)
testing_data = testing_data.map(remove_special_characters)

Map:   0%|          | 0/9251 [00:00<?, ? examples/s]

Map:   0%|          | 0/2255 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(training_data.remove_columns(["path","audio"]))

,sentence
0,huwezi kufanya kazi hiyo yote
1,congo ilishinda kikombe kwenye fainali zilizofanyika cameroon
2,elfu mbili kumi na saba ambapo ni sawa na
3,isipokuwa sehemu za kaskazini za urusi huhesabiwa mara nyingi upande wa ulaya ya kaskazini
4,badala ya kwenda alipotoka alienda kwenye mafaili ya kumbukumbu za ardhi
5,ategemea sana mahitaji ya nchi husika
6,hata hivyo wakisitisha matibabu viwango vya virusi vinaongezeka na vinaweza kuonekana katika damu yake
7,muda wote sasa yako haitabiriki imefichika maana
8,serikali inapaswa kutumia vituo vikubwa vya runinga vya nje ya nchi
9,wamasai huamini kwamba mungu aliwapa ngombe wote duniani


In [ ]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
    return batch

In [ ]:
training_data = training_data.map(replace_hatted_characters)
testing_data = testing_data.map(replace_hatted_characters)

Map:   0%|          | 0/9251 [00:00<?, ? examples/s]

Map:   0%|          | 0/2255 [00:00<?, ? examples/s]

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = training_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=training_data.column_names)
vocab_test = testing_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=testing_data.column_names)


Map:   0%|          | 0/9251 [00:00<?, ? examples/s]

Map:   0%|          | 0/2255 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 '(': 1,
 ')': 2,
 '/': 3,
 '_': 4,
 'a': 5,
 'b': 6,
 'c': 7,
 'd': 8,
 'e': 9,
 'f': 10,
 'g': 11,
 'h': 12,
 'i': 13,
 'j': 14,
 'k': 15,
 'l': 16,
 'm': 17,
 'n': 18,
 'o': 19,
 'p': 20,
 'q': 21,
 'r': 22,
 's': 23,
 't': 24,
 'u': 25,
 'v': 26,
 'w': 27,
 'x': 28,
 'y': 29,
 'z': 30,
 'ã': 31,
 'å': 32,
 'ï': 33,
 'ø': 34,
 'ː': 35,
 'μ': 36,
 '’': 37,
 '•': 38}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

41

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [ ]:
repo_name = "sw_asr"

In [ ]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/mussacharles60/sw_asr/commit/2a8c734e41115123493c597474b6e2074eced13d', commit_message='Upload tokenizer', commit_description='', oid='2a8c734e41115123493c597474b6e2074eced13d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [ ]:
training_data[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/4a443506d39b8b48fcc983dd5b38f1616acba3909b5a917d6302bbbe14aec72e/sw_train_0/common_voice_sw_28581540.mp3'

In [ ]:
training_data[0]['audio']

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/4a443506d39b8b48fcc983dd5b38f1616acba3909b5a917d6302bbbe14aec72e/sw_train_0/common_voice_sw_28581540.mp3',
 'array': array([-1.42108547e-14, -1.56319402e-13, -5.82645043e-13, ...,
         2.36777602e-08,  1.87174543e-08,  8.03334554e-09]),
 'sampling_rate': 48000}

In [ ]:
# reduce the audio sample rate from 48KHz to 16KHz as it needed for training our model

training_data = training_data.cast_column("audio", Audio(sampling_rate=16_000))
testing_data = testing_data.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
# preview the converted sample rate

training_data[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/4a443506d39b8b48fcc983dd5b38f1616acba3909b5a917d6302bbbe14aec72e/sw_train_0/common_voice_sw_28581540.mp3',
 'array': array([-9.60426405e-10, -2.44472176e-09,  0.00000000e+00, ...,
         3.43425199e-09,  2.18569767e-08,  2.17551133e-08]),
 'sampling_rate': 16000}

In [ ]:
# preview one random audio from the training data

import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(training_data)-1)

print(training_data[rand_int]["sentence"])
ipd.Audio(data=training_data[rand_int]["audio"]["array"], autoplay=True, rate=16000)


baada ya pale jacob alianza kutoa maelekezo juu ya muuaji wa wale watoto wawili


In [ ]:
rand_int = random.randint(0, len(training_data)-1)

print("Target text:", training_data[rand_int]["sentence"])
print("Input array shape:", training_data[rand_int]["audio"]["array"].shape)
print("Sampling rate:", training_data[rand_int]["audio"]["sampling_rate"])


Target text: mabaya ya madaraka takwimu yake ni tata na kukaa katika nafasi ya kuwajibika na
Input array shape: (82368,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


In [ ]:
training_data = training_data.map(prepare_dataset, remove_columns=training_data.column_names)
testing_data = testing_data.map(prepare_dataset, remove_columns=testing_data.column_names)

Map:   0%|          | 0/9251 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2255 [00:00<?, ? examples/s]

In [ ]:
#max_input_length_in_sec = 5.0
#common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [ ]:
wer_metric = load_metric("wer")


<ipython-input-56-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1913: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=200,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=training_data,
    eval_dataset=testing_data,
    tokenizer=processor.feature_extractor,
)


In [ ]:
# train

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,7.154200,inf,1.000000
200,2.877900,inf,1.000000
300,1.791900,inf,0.711384
400,0.615200,inf,0.555385
500,0.476200,inf,0.485400
600,0.403200,inf,0.435992
700,0.305900,inf,0.392986
800,0.301000,inf,0.370239
900,0.263200,inf,0.345747
1000,0.208700,inf,0.332628


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran

TrainOutput(global_step=2890, training_loss=0.5701992962187137, metrics={'train_runtime': 8711.5781, 'train_samples_per_second': 10.619, 'train_steps_per_second': 0.332, 'total_flos': 1.545296582709629e+19, 'train_loss': 0.5701992962187137, 'epoch': 9.98})

In [ ]:
# save to huggingface's custom model repo

trainer.push_to_hub()


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

events.out.tfevents.1699138553.52a200ca3a05.1368.0:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

'https://huggingface.co/mussacharles60/sw_asr/tree/main/'

In [ ]:
# share model

from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained('mussacharles60/' + repo_name)
processor = Wav2Vec2Processor.from_pretrained('mussacharles60/' + repo_name)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# evaluate model
# load the model and verify that it indeed has learned to transcribe Swahili speech.

# first load the pretrained checkpoint

# model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
# processor = Wav2Vec2Processor.from_pretrained(repo_name)

model = model.to("cuda")

In [ ]:
input_dict = processor(testing_data[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
# load testing dataset to test our model

# common_voice_test_transcription = load_dataset("common_voice", "sw", data_dir="./cv-corpus-15.0-2023-09-08-sw", split="test")
common_voice_test_transcription = load_dataset('mozilla-foundation/common_voice_13_0', 'sw', split='test')


In [ ]:
common_voice_test_transcription

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 11271
})

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[0]["sentence"].lower())


Prediction:
wachamvuzi wa soka wanamkada meesi kama nyota hatari zaidi duniani

Reference:
wachambuzi wa soka wanamtaja messi kama nyota hatari zaidi duniani


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# copy sw_asr generated folder to google drive
!cp -r /content/sw_asr /content/drive/MyDrive/mcv_swahili/sw_asr_model_v1